In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = set(stopwords.words('english'))
from sklearn.metrics import classification_report, confusion_matrix
import os
from tqdm import tqdm
tqdm.pandas()
from collections import Counter

import os, pathlib, shutil, random
import tensorflow as tf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


## Download IMDB Data

In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  20.1M      0  0:00:03  0:00:03 --:--:-- 20.1M


In [5]:
# read data
# data = pd.read_csv('data/IMDB Dataset.csv')
# data.head()

## Prepare data

### Creating and organizing files in folders

In [3]:
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)



### Load train, validation and test datasets from folders

In [4]:
train_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [5]:
text_only_train_ds = train_ds.map(lambda x, y: x)

In [6]:
counter = 0
for x, y in train_ds:
    for text, label in zip(x,y):
        counter += 1
        print("Review :", text)
        print("Label: ", label)
        print("\n")
        if counter > 5:
            break
    break

Review : tf.Tensor(b'I found this movie hilarious. The spoofs on other popular movies of that time were some of the funniest I have seen in this sort of movie. Give it a try. If you saw the movies that this movie is spoofing, and you get the humor, you should enjoy the movie.<br /><br />I (and the others who watched the movie with me) felt the funniest part in the movie (this is not a spoiler because I will NOT tell you what actually happens) was a scene with the "flashy thingy" from MIB. When they first discover the device and do not know what it is does... and then again later in the movie... you\'ll understand when you get there.<br /><br />My only complaint about the movie is that I have never been able to find it in DVD so that I could buy a copy.', shape=(), dtype=string)
Label:  tf.Tensor(1, shape=(), dtype=int32)


Review : tf.Tensor(b"The Sensuous Nurse (1975) was a Italian sexual comedy that starred the one time Bond girl Ursula Andress. Man was she hot in this movie.. She wa

### Convert text data into numeric

In [7]:


max_length = 600
max_tokens = 20000
text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)




In [8]:
counter = 0
for x, y in int_train_ds:
    for text, label in zip(x,y):
        counter += 1
        print("Review :", text)
        print("Label: ", label)
        print("\n")
        if counter >= 1:
            break
    break

Review : tf.Tensor(
[  10 1980   58  145    3 1940   11 1147  169  332   10  152  103   35
   59   27  478    6  187    4  362  559  937 5693 1147    3   94    9
    4   18   35  116    9   16 1462 6794 2878  160  538 5229 1129   18
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0  

### A sequence model built on one-hot encoded vector sequences

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.one_hot(x, depth=max_tokens)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])


In [10]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])


In [20]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1)

625/625 [==============================] - 37s 59ms/step - loss: 0.2867 - accuracy: 0.8963 - val_loss: 0.3848 - val_accuracy: 0.8630


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None, 600, 20000)        0         
                                                                 
 bidirectional (Bidirection  (None, 64)                5128448   
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [1]:
## Try with embedding layer instead of one-hot encoding

In [14]:
embedding_layer = tf.keras.layers.Embedding(input_dim=max_tokens, output_dim=256)

### Build Model (Embedding from scratch)

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])


In [22]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [23]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=2)

Epoch 1/2
625/625 [==============================] - 103s 150ms/step - loss: 0.4366 - accuracy: 0.7940 - val_loss: 0.3233 - val_accuracy: 0.8658
Epoch 2/2
625/625 [==============================] - 62s 99ms/step - loss: 0.2806 - accuracy: 0.8877 - val_loss: 0.3032 - val_accuracy: 0.8766


In [ ]:
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

### Use pretrained embedding instead of building from scratch

In [24]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-09-10 06:53:09--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-09-10 06:53:09--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-09-10 06:53:09--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [25]:
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


### prepare Glove word Embedding matrix

In [26]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [27]:
embedding_layer = tf.keras.layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

## Build Model with pretrained Embedding

In [28]:
model = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [29]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [30]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)

Epoch 1/10
625/625 [==============================] - 42s 53ms/step - loss: 0.5705 - accuracy: 0.6996 - val_loss: 0.4934 - val_accuracy: 0.7740
Epoch 2/10
625/625 [==============================] - 28s 45ms/step - loss: 0.4603 - accuracy: 0.7897 - val_loss: 0.4053 - val_accuracy: 0.8208
Epoch 3/10
625/625 [==============================] - 27s 44ms/step - loss: 0.4091 - accuracy: 0.8170 - val_loss: 0.4034 - val_accuracy: 0.8080
Epoch 4/10
625/625 [==============================] - 31s 49ms/step - loss: 0.3772 - accuracy: 0.8362 - val_loss: 0.3579 - val_accuracy: 0.8440
Epoch 5/10
625/625 [==============================] - 34s 54ms/step - loss: 0.3557 - accuracy: 0.8486 - val_loss: 0.3621 - val_accuracy: 0.8388
Epoch 6/10
625/625 [==============================] - 28s 44ms/step - loss: 0.3313 - accuracy: 0.8602 - val_loss: 0.3193 - val_accuracy: 0.8644
Epoch 7/10
625/625 [==============================] - 28s 44ms/step - loss: 0.3136 - accuracy: 0.8693 - val_loss: 0.3159 - val_accuracy:

In [31]:
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 16s 20ms/step - loss: 0.3368 - accuracy: 0.8658
Test acc: 0.866


In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                34048     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2034113 (7.76 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 2000000 (7.63 MB)
_________________________________________________________________
